<h1>Process Reviews </h1>

Want to predict rating from the textual review content. We need to ingest over 12 million reviews, then filter on them.

In [2]:
import pandas as pd
import numpy as np 
import dask.dataframe as dd

In [3]:
file_path = './data/review-Pennsylvania_10.json'

In [7]:
from dask.distributed import Client 

client = Client(memory_limit = '4GB', n_workers=2, threads_per_worker=2) 
print(client.dashboard_link)

/Users/alanwu/Documents/GitHub/Google-Local-Review-Analysis/venv/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52245 instead
  warnings.warn(


http://127.0.0.1:52245/status


### Ingesting reviews

In [20]:
import gzip
import json

df = dd.read_json(file_path, lines=True, blocksize="16MB")

df.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.049447e+20,Jaron Whitfield,1517731762839,5,"Joe is quite unique of his line of work, he as...",<NA>,<NA>,0x89c6c63c8cd87141:0x54d0d283872eecbb
1,1.176097e+20,Jonathan McCarthy,1476276291163,5,For such a small place their impact on my life...,<NA>,<NA>,0x89c6c63c8cd87141:0x54d0d283872eecbb
2,1.105632e+20,Rocky Kev,1338826945578,5,I usually give them a call before I stop by to...,<NA>,<NA>,0x89c6c63c8cd87141:0x54d0d283872eecbb
3,1.128960e+20,Josep Valls,1363286110554,5,My bike had been sitting outdoors for a good w...,<NA>,<NA>,0x89c6c63c8cd87141:0x54d0d283872eecbb
4,1.106197e+20,Timaree Schmit,1548798329760,5,Always an easy experience. Service is knowledg...,<NA>,<NA>,0x89c6c63c8cd87141:0x54d0d283872eecbb


In [15]:
print(f"Number of reviews {df.shape[0].compute()}")

Number of reviews 12772358


### drop the pics, resp

In [21]:
df = df.drop(columns=['resp', 'pics'])
df.head()

,user_id,name,time,rating,text,gmap_id
0,1.049447e+20,Jaron Whitfield,1517731762839,5,"Joe is quite unique of his line of work, he as...",0x89c6c63c8cd87141:0x54d0d283872eecbb
1,1.176097e+20,Jonathan McCarthy,1476276291163,5,For such a small place their impact on my life...,0x89c6c63c8cd87141:0x54d0d283872eecbb
2,1.105632e+20,Rocky Kev,1338826945578,5,I usually give them a call before I stop by to...,0x89c6c63c8cd87141:0x54d0d283872eecbb
3,1.128960e+20,Josep Valls,1363286110554,5,My bike had been sitting outdoors for a good w...,0x89c6c63c8cd87141:0x54d0d283872eecbb
4,1.106197e+20,Timaree Schmit,1548798329760,5,Always an easy experience. Service is knowledg...,0x89c6c63c8cd87141:0x54d0d283872eecbb


In [22]:
summary_stats = df.describe().compute()
summary_stats

,user_id,time,rating
count,1.277236e+07,1.277236e+07,1.277236e+07
mean,1.092328e+20,1.063160e+11,4.293200e+00
std,5.318876e+18,3.926447e+10,1.062594e+00
min,1.000000e+20,6.626016e+11,1.000000e+00
25%,1.057053e+20,1.530289e+12,4.000000e+00
50%,1.096047e+20,1.561359e+12,5.000000e+00
75%,1.140734e+20,1.597756e+12,5.000000e+00
max,1.184467e+20,1.631133e+12,5.000000e+00


remove nulls

In [23]:
missing_vals = df.isnull().sum().compute() 
missing_vals

user_id          0
name             0
time             0
rating           0
text       5965398
gmap_id          0
dtype: int64

In [24]:
df = df.dropna()

In [25]:
df.to_parquet('./data/reviews.parquet', engine='pyarrow')